# FUNZIONI DASK_CUDF CHE NON VANNO (PER FARE CONFRONTO)

## Questa è la sezione da portare per far vedere i limiti di dask_cudf

### Conversione colonna da stringa a datetime

Errore che restituisce:
"TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy()."

In [ ]:
#CONVERSIONE COLONNE SU DASK_CUDF 
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}
cols = {
    0:'day',
    1:'month',
    2:'year'
}
a = base.split('gas_start_date', 2)
a = a.rename(columns=cols)
a['month'] = a['month'].str.lower().map(mesi)
dd.to_datetime(a).head()

### Conversione colonna da stringa in formato "1,22 unita_misura" a float

In [ ]:
#Ottenere unità di misura
misure = base.split('average_gas_bill_cost', 1)[1].value_counts().index.compute()[0]
#rimozione unità di misura
base.strip(['average_gas_bill_cost'], '€/smc')['average_gas_bill_cost']
#sostituzione della , con .
base.replace(['average_gas_bill_cost'], ',', '.', regex=True)['average_gas_bill_cost']

#### Le operazioni preliminari vanno a buon fine, qua si aprono più scenari (fallimentari)

questa soluzione è quella corretta su dask normale, utilizzando dask_cudf però genera un errore dovuto alla conversione implicita di un array in un numpy array, cosa non fattibile all'interno della GPU quindi consiglia di costruire esplicitamente una matrice tramite Cupy (Cuda Python) ma queste sono operazioni interne alla libreria

In [ ]:
dd.to_numeric(base.get_df()['average_gas_bill_cost'])
 

Qua da 2 diverse tipologie di errori: una dovuta ai metadati e l'altra alla presenza di valori non convertibili, anche togliedo tutti i valori nan con [~base['average_gas_bill_cost'].isna()] la situazione non cambia

In [ ]:
base.get_df()['average_gas_bill_cost'].map_partitions(cudf.to_numeric, 'raise')

# Pipeline

In [ ]:
''' 
+++++ UTILITY ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code for the customer that owns this utility
    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized
    city --> String --> City where the utility is located
    address --> String --> (Anonymized) address of the utility location
'''

In [ ]:
''' 
+++++ CUSTOMER ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code that identifies the customer
    nominative --> String --> (Anonymized) customer name
    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)
    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18
'''

In [ ]:
''' 
+++++ INVOICE ATTRIBUTES +++++
    
    bill_id --> Int --> Invoice identifier
    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot
    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot
    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot 
    date --> Date --> Start date
    light_start_date --> Date --> Start date of electricity invoice
    light_end_date --> Date --> End date of electricity invoice
    tv --> Float --> Television fee to pay
    gas_amount --> Float --> Gas fee to pay
    gas_average_cost --> Float --> Average cost of gas
    light_average_cost --> Float --> Average cost of electricity
    emission_date --> Date --> Emission date
    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)
    gas_start_date --> Date --> Start date of gas invoice
    gas_end_date --> Date --> End date of gas invoice
    extra_fees --> Float --> Extra fees to pay
    gas_consumption --> Float --> Consumed gas
    light_consumption --> Float --> Consumed electricity
    gas_offer --> Float --> Name of the subscribed gas plan (anonymized)
    light_offer_type --> String --> Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
    light_offer --> String --> Name of the subscribed electricity plan (anonymized)
    total_amount --> Float --> gas_amount + light_amount + extra_fees
    howmuch_pay --> Float --> Overall amount to pay, computed as total_amount + tv
    light_amount --> Float --> Amount to pay for the electricity
    average_unit_light_cost --> Float --> Average cost for electricity
    average_light_bill_cost --> Float --> Average cost for the electricity invoice
    average_unit_gas_cost --> Float --> Average cost for gas
    average_gas_bill_cost --> Float --> Average cost for the gas invoice
    billing_frequency --> String --> Billing frequency (‘monthly’, ‘quarterly’, etc.)
    bill_type --> String --> Kind of invoice (False means a “standard bill”)
    gas_system_charges --> Float --> Extra gas fees
    light_system_charges --> Float --> Extra electricity fees
    gas_material_cost --> Float --> Costs for gas
    light_transport_cost --> Float --> Extra electricity fees
    gas_transport_cost --> Float --> Extra gas fees
    light_material_cost --> Float --> Costs for electricity
'''

In [ ]:
''' 
+++++ DATA INGESTION +++++
    • Read data from its source
    • Study how to deal with data that doesn’t fit in memory (!) --> Da chiede a Gagliardelli
    • Locate missing values
    • Locate outliers
    • Sort data
'''

# Loading Dataset

In [1]:
import dask_cudf as dc
import dask.dataframe as dd
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import cudf
import numpy as np
#import graphviz
from base_dask_cudf import BaseDfBench
base = BaseDfBench("DASK")
# base = BaseDfBench()

In [2]:
# READ DATA FROM ITS SOURCE
# --- remove _sample one day

# We can use bill_id as the index for our dataset --> meglio di no
# base.load_dataset('/data/invoices_sample.csv', 'csv', index_col='bill_id')

# Uncomment the line under this one to load the pre-converted dataset
base.load_dataset('parquet', path='data/mod_parquet')
# base.load_dataset('parquet')
df = base.get_df()
columns = base.get_columns()
base.npartitions()

57

In [3]:
# Dataset's rows
# I commented the real computation because it takes a lot of time
# dataset_rows = base.get_df().shape[0].compute()
dataset_rows = 10497143
dataset_rows

10497143

In [ ]:
#verica di user_code+customer_code = PK utility
#PK customer = user_code
#PK invoice = bill_id
df.groupby(by=['user_code', 'customer_code'])['bill_id'].count().compute()

## Load big Dataset

In [ ]:
# STUDY HOW TO DEAL WITH DATA THAT DON'T FIT IN MEMORY (!)
# Bella storia
!du -sh '/data/invoices_sample.csv'

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

In [ ]:
cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),
)
client = Client(cluster)
client

In [ ]:
%%time
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="1 GiB")
print(f"DF partitions: {ddf.npartitions}")
print(f"DF: {ddf}")

# UTILS

In [15]:
# CONVERTING ALL THE STRING COLUMNS TO OBJECT COLUMN
# THIS BECAUSE STRING COLUMNS CAUSE A LOT OF ERRORS WHEN PROCESSED AS STRINGS
for k,v in base.get_columns_types().items():
    if v == 'string':
        base[k] = base[k].astype('object')
    elif v == 'Float64':
        base[k] = base[k].astype('float')

In [14]:
# Seguendo il GroundTruth mystico
mappone = {'bill_id': 'int64',
 'F1_kWh': 'float64',
 'F2_kWh': 'float64',
 'F3_kWh': 'float64',
 'city': 'object',
 'address': 'object',
 'nominative': 'object',
 'sex': 'object',
 'age': 'int64',
 'user_code': 'object',
 'date': 'datetime64[ns]',
 'light_start_date': 'datetime64[ns]',
 'light_end_date': 'datetime64[ns]',
 'tv': 'float64',
 'gas_amount': 'float64',
 'gas_average_cost': 'float64',
 'light_average_cost': 'float64',
 'emission_date': 'datetime64[ns]',
 'supply_type': 'object',
 'gas_start_date': 'datetime64[ns]',
 'gas_end_date': 'datetime64[ns]',
 'extra_fees': 'float64',
 'gas_consumption': 'float64',
 'light_consumption': 'float64',
 'gas_offer': 'float64',
 'light_offer_type': 'object',
 'light_offer': 'object',
 'howmuch_pay': 'float64',
 'total_amount': 'float64',
 'light_amount': 'float64',
 'average_unit_light_cost': 'float64',
 'average_light_bill_cost': 'float64',
 'average_unit_gas_cost': 'float64',
 'average_gas_bill_cost': 'float64',
 'customer_code': 'object',
 'billing_frequency': 'object',
 'bill_type': 'object',
 'gas_system_charges': 'float64',
 'light_system_charges': 'float64',
 'gas_material_cost': 'float64',
 'light_transport_cost': 'float64',
 'gas_transport_cost': 'float64',
 'light_material_cost': 'float64'
}

In [15]:
#Check sui tipi di dato attuali
type_dict = base.get_columns_types()
for k,v in type_dict.items():
    if v != mappone[k]:
        print(f'mismatch in {k} with values: {v}, {mappone[k]}')

mismatch in tv with values: object, float64
mismatch in extra_fees with values: object, float64
mismatch in gas_offer with values: uint64, float64


In [6]:
# Trovare le tipologie di colonne a seconda della tipologia
int_cols = base.col_type(find=['int'])
string_cols = base.col_type(find=['string']) #object
float_cols = base.col_type(find=['float'])
numeric_cols = base.col_type(find=['numeric'])
date_cols = base.col_type(find=['date'])

# Convert Columns

## Da portare

In [ ]:
# DONE gas_amount, howmuch_pay, total_amount, light_amount to float
# DONE gas_average_col, gas_offer, average_unit_light_cost, average_light_bill_cost, average_unit_gas_cost, average_gas_bill_cost deve essere float non int, basta un astype()
# light_offer è segnato come string perché dovrebbe essere un nome ma nel dataset è un int (what)
# il caso light_offer risolto: in teoria è str ma in pratica è stato anonimizzato, provo il cast to string

In [6]:
base['light_offer'] = base['light_offer'].astype(str)

In [9]:
len(base.col_type('string'))

28

### Date conversion

In [10]:
#CONVERSIONE COLONNE SU DASK_CUDF 
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}
cols = {
    0:'day',
    1:'month',
    2:'year'
}
#finding columns with date keyword
date_cols = []
for c in df.columns:
    # print(base[c].dtype)
    if 'date' in c.split('_') and base[c].dtype != 'datetime64[ns]':
        date_cols.append(c)

print(f'columns to convert to date: {date_cols}')
#converting these columns
for c in date_cols:

    print(c , df[c].dtype)
    
    a = base.split(c, 2)
    a = a.rename(columns=cols)
    a['month'] = a['month'].str.lower().map(mesi)
    base.get_df()[c] = dd.to_datetime(a, format="%d/%m/%Y")
    # print(a.head())
    
    #df[c] = base.split(c, 2)
    
    # print(base[c].head())

columns to convert to date: ['date', 'light_start_date', 'light_end_date', 'emission_date', 'gas_start_date', 'gas_end_date']
date object
light_start_date object
light_end_date object
emission_date object
gas_start_date object
gas_end_date object


In [13]:
base.df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=57,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,Float64,Float64,Float64,object,object,object,object,int64,object,datetime64[ns],datetime64[ns],datetime64[ns],object,object,object,Float64,datetime64[ns],object,datetime64[ns],datetime64[ns],object,Float64,Float64,uint64,object,object,object,object,object,object,object,object,object,object,object,object,Float64,Float64,Float64,Float64,Float64,Float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Removing measure unit from strings and converting them into numbers

In [12]:
# this line can be used to retrieve alle the columns with a number pattern inside them but is too resources consuming
# Due to the small number of columns We preferred to analize them one by one

# base['howmuch_pay'].str.contains('[0-9]+[.,][0-9]*', regex=True).count().compute()
# [0-9]+[.,][0-9]*

In [4]:
#finding string columns with number inside
to_numeric_cols = []
for c in df.columns:
    if ('average'  in c.split('_') or 'amount' in c.split('_')) and (base[c].dtype in ['object', 'str']):
        # print(base[c].dtype)
        to_numeric_cols.append(c)

# Added manually after column analysis 
to_numeric_cols.append('howmuch_pay')
to_numeric_cols.append('tv')
to_numeric_cols.append('extra_fees')
print(to_numeric_cols)

['howmuch_pay', 'tv', 'extra_fees']


In [5]:
to_numeric_cols = to_numeric_cols[1:]
to_numeric_cols

['tv', 'extra_fees']

In [14]:
# Exctracting measure unit from the column
# Commented bc is time consuming
# base.split('average_gas_bill_cost', 1)[1].value_counts().index.compute()[0]

In [6]:
# Converting columns from string tu numeric
partitions = base.npartitions()
for c in to_numeric_cols:
    base[c] = base[c].str.extract(r'([0-9]+[,.][0-9]*)', flags=0, expand=False)
    # print(f'{c} measure: {measure}')
    base.replace([c], ',', '.', regex=True)
    base[c] = dd.to_numeric(base[c], errors='coerce', meta=('float'))
# base.get_df().head()['average_gas_bill_cost']

In [7]:
base.df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=57,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,Float64,Float64,Float64,object,object,object,object,int64,object,datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,Float64,datetime64[ns],object,datetime64[ns],datetime64[ns],float64,Float64,Float64,uint64,object,object,float64,float64,float64,float64,float64,float64,float64,object,object,object,Float64,Float64,Float64,Float64,Float64,Float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
# Strip all the columns in order to remove all the whitespaces 
base.strip(string_cols)

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=57,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,Float64,Float64,Float64,object,object,object,object,int64,object,datetime64[ns],datetime64[ns],datetime64[ns],object,float64,float64,Float64,datetime64[ns],object,datetime64[ns],datetime64[ns],object,Float64,Float64,uint64,object,object,float64,float64,float64,float64,float64,float64,float64,object,object,object,Float64,Float64,Float64,Float64,Float64,Float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
base.df = base.df.persist()

In [18]:
base['howmuch_pay'].head()

0      0.00
1     47.22
2      0.00
3     58.53
4    383.66
Name: howmuch_pay, dtype: float64

### Save the modified dataset

In [9]:
# Save the converted dataset
base.to_parquet('data/mod_parquet')

In [10]:
base.load_dataset('parquet', path='data/mod_parquet')
base.get_columns_types()

{'bill_id': 'int64',
 'F1_kWh': 'Float64',
 'F2_kWh': 'Float64',
 'F3_kWh': 'Float64',
 'city': 'object',
 'address': 'object',
 'nominative': 'object',
 'sex': 'object',
 'age': 'int64',
 'user_code': 'object',
 'date': 'datetime64[ns]',
 'light_start_date': 'datetime64[ns]',
 'light_end_date': 'datetime64[ns]',
 'tv': 'object',
 'gas_amount': 'float64',
 'gas_average_cost': 'float64',
 'light_average_cost': 'Float64',
 'emission_date': 'datetime64[ns]',
 'supply_type': 'object',
 'gas_start_date': 'datetime64[ns]',
 'gas_end_date': 'datetime64[ns]',
 'extra_fees': 'object',
 'gas_consumption': 'Float64',
 'light_consumption': 'Float64',
 'gas_offer': 'uint64',
 'light_offer_type': 'object',
 'light_offer': 'object',
 'howmuch_pay': 'float64',
 'total_amount': 'float64',
 'light_amount': 'float64',
 'average_unit_light_cost': 'float64',
 'average_light_bill_cost': 'float64',
 'average_unit_gas_cost': 'float64',
 'average_gas_bill_cost': 'float64',
 'customer_code': 'object',
 'billing

## Da non portare

In [ ]:
# Dovrebbe accorgersi di colonne con similarità > soglia
# TODO --> FUNZIONE DA RIFARE PERCHé USA I SET E QUINDI NON CONFRONTA RIGA-RIGA 
#          MA SET(ELEMENTI_COL1) CON SET(eLEMENTI_COL2)
def get_duplicate_col(df, soglia=0.9):
        """
        Return a list of duplicate columns, if exists.
        Duplicate columns are those which have same values for each row.
        """  
        cols = df.columns.values
        #return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if self.df[cols[i]].equals(self.df[cols[j]])]
        
        return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if len(set(cols[i]) & set(cols[j])) / float(len(set(cols[i]) | set(cols[j]))) > soglia ]

# 1. INGESTION & DISCOVERY

## Analysis DA PORTARE

In [7]:
#esempio
base.locate_null_values('address').head()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
1611,1611,23311.0,2219.0,467.0,VIETRI DI POTENZA,None,None,P,0,8CE314D06FEFFD54C0F1497C20C479F376102747,...,NaN,F631ED0EF8EC71428366F15C369AA53792A6F20D,monthly,False,<NA>,1176.31,<NA>,481.81,<NA>,2067.89
1627,1627,210.0,149.0,226.0,VIETRI DI POTENZA,None,None,P,45,A1E7C992D46A2D6D911C3595DF603D3F2C6BEB37,...,NaN,1E9BC1B19300B6E472980894714A4C8382C565EA,monthly,False,<NA>,76.08,<NA>,50.45,<NA>,85.57
1639,1639,95.0,52.0,105.0,VIETRI DI POTENZA,None,None,P,0,3CE203296BE19C1257020DA7EFEB84A0D3F4F7B7,...,NaN,CC5C0DB0AA0034008B3398775B1EC89FC21A9DDB,monthly,False,<NA>,40.8,<NA>,29.47,<NA>,45.65
1858,1858,65.0,34.0,60.0,VIETRI DI POTENZA,None,None,P,66,D3117D65827932EB6A7F54F73EB1700C0D9F289C,...,NaN,EB24C1C62642D39E65FB5BA50044153D2BF6EEC9,monthly,False,<NA>,44.17,<NA>,36.34,<NA>,47.11
2511,2511,285.0,406.0,232.0,VIETRI DI POTENZA,None,None,P,0,44F5D3195791957FCF7010485404E83F3BDB9F0B,...,NaN,1D2EAAAFDD212B85E05CC2275D44020B05B1E5B7,monthly,False,<NA>,101.28,<NA>,74.09,<NA>,87.2


## Locate Missing Values DA PORTARE

In [28]:
# LOCATE MISSING VALUES
#rows = df.shape[0]
#wait(ddf.shape[0])
#print("Total rows", rows)

# These are the colums with at least one missing value
column_count_nan = {}
# compute the presence of null value on time instead every iteration for performance reasons
na = base.get_df().isna()
val = na.any().compute()

for c, val in val.to_pandas().items():
    count = na[c].sum().compute()
    print(c, count)
    if (val):
        #print(c, "\t Affected rows:", count)
        column_count_nan[c] = count
    else:
        #print(c, "\t Qui no missing values")
        column_count_nan[c] = count

bill_id 0
F1_kWh 166
F2_kWh 92
F3_kWh 689
city 15
address 2760
nominative 2760
sex 0
age 0
user_code 0
date 7871
light_start_date 7871
light_end_date 7871
tv 0
gas_amount 3686754
gas_average_cost 4287486
light_average_cost 5967463
emission_date 0
supply_type 0
gas_start_date 7871
gas_end_date 7871
extra_fees 0
gas_consumption 3686753
light_consumption 5686817
gas_offer 0
light_offer_type 0
light_offer 0
howmuch_pay 0
total_amount 0
light_amount 5686804
average_unit_light_cost 5967414
average_light_bill_cost 5967119
average_unit_gas_cost 4287486
average_gas_bill_cost 4285780
customer_code 0
billing_frequency 784810
bill_type 0
gas_system_charges 3762487
light_system_charges 5741641
gas_material_cost 3755162
light_transport_cost 5740814
gas_transport_cost 3761775
light_material_cost 5740821


In [32]:
# import json

# class NpEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         if isinstance(obj, np.floating):
#             return float(obj)
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return super(NpEncoder, self).default(obj)
    
# with open('data/initial_null_val.json', 'w+') as f:
#     # this would place the entire output on one line
#     # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
#     json.dump(column_count_nan, f, cls=NpEncoder)

In [29]:
column_with_nan = []

perc = 1
soglia = ((perc * base.get_df().shape[0]) / 100).compute()
print(soglia)

for c, val in column_count_nan.items():
    #print(f'{c}')
    if val > soglia:
        print(f'{c} val: {val}')
        column_with_nan.append(c)
        
column_with_nan

# Si può notare come nessuna colonna abbia il 100% di valori nulli, ma ci sono altresì tante 
# colonne con un numero elevato di nan. Poche colonne hanno un numero basso di nan.

104971.43
gas_amount val: 3686754
gas_average_cost val: 4287486
light_average_cost val: 5967463
gas_consumption val: 3686753
light_consumption val: 5686817
light_amount val: 5686804
average_unit_light_cost val: 5967414
average_light_bill_cost val: 5967119
average_unit_gas_cost val: 4287486
average_gas_bill_cost val: 4285780
billing_frequency val: 784810
gas_system_charges val: 3762487
light_system_charges val: 5741641
gas_material_cost val: 3755162
light_transport_cost val: 5740814
gas_transport_cost val: 3761775
light_material_cost val: 5740821


['gas_amount',
 'gas_average_cost',
 'light_average_cost',
 'gas_consumption',
 'light_consumption',
 'light_amount',
 'average_unit_light_cost',
 'average_light_bill_cost',
 'average_unit_gas_cost',
 'average_gas_bill_cost',
 'billing_frequency',
 'gas_system_charges',
 'light_system_charges',
 'gas_material_cost',
 'light_transport_cost',
 'gas_transport_cost',
 'light_material_cost']

### Analisi extra (utile estenderla all'intero dataset)

In [6]:
#Visto che address e nominative hanno lo stesso numero di NA, vale la 
#pena controllare ed eventualmente eliminare le righe?

base.locate_null_values('nominative').shape[0].compute()

2760

In [7]:
base.locate_null_values('address').shape[0].compute()

2760

In [9]:
df[df['city'] == 'VIETRI DI POTENZA'].compute()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
1611,1611,23311.0,2219.0,467.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,8CE314D06FEFFD54C0F1497C20C479F376102747,...,<NA>,F631ED0EF8EC71428366F15C369AA53792A6F20D,monthly,False,<NA>,1176.31,<NA>,481.81,<NA>,2067.89
1627,1627,210.0,149.0,226.0,VIETRI DI POTENZA,<NA>,<NA>,P,45,A1E7C992D46A2D6D911C3595DF603D3F2C6BEB37,...,<NA>,1E9BC1B19300B6E472980894714A4C8382C565EA,monthly,False,<NA>,76.08,<NA>,50.45,<NA>,85.57
1639,1639,95.0,52.0,105.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,3CE203296BE19C1257020DA7EFEB84A0D3F4F7B7,...,<NA>,CC5C0DB0AA0034008B3398775B1EC89FC21A9DDB,monthly,False,<NA>,40.8,<NA>,29.47,<NA>,45.65
1858,1858,65.0,34.0,60.0,VIETRI DI POTENZA,<NA>,<NA>,P,66,D3117D65827932EB6A7F54F73EB1700C0D9F289C,...,<NA>,EB24C1C62642D39E65FB5BA50044153D2BF6EEC9,monthly,False,<NA>,44.17,<NA>,36.34,<NA>,47.11
2511,2511,285.0,406.0,232.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,44F5D3195791957FCF7010485404E83F3BDB9F0B,...,<NA>,1D2EAAAFDD212B85E05CC2275D44020B05B1E5B7,monthly,False,<NA>,101.28,<NA>,74.09,<NA>,87.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535993,2506364,154.0,130.0,231.0,VIETRI DI POTENZA,<NA>,<NA>,M,0,B604345ABA6914CC140753F1C43DD5F22E84C0F2,...,<NA>,ACFABF3A26CB1D3BAE16D631A591A68E0632648D,monthly,False,<NA>,45.78,<NA>,24.22,<NA>,102.87
536213,2506584,328.0,106.0,213.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,5DE7A59C94526D1AE0A9B2F677FB8A7E9392F05D,...,<NA>,E5FDCA4AAE441EDD18E252195A15B0311668CA83,monthly,False,<NA>,48.95,<NA>,21.77,<NA>,84.41
536734,2507105,887.0,308.0,612.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,A28B8B991F008ADDAB2CB72265E4471BB4100A3A,...,<NA>,2BB9E9903163EA9EE0827CEF7C226FE96E309DAA,monthly,False,<NA>,109.07,<NA>,32.8,<NA>,192.82
536750,2507121,1823.0,1336.0,3100.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,E700B1E71861717A167A2F37A076D570E196F73F,...,<NA>,7ED689C6B0A04EE3419416F6AD849E4766337115,monthly,False,<NA>,187.52,<NA>,46.56,<NA>,372.28


In [12]:
# PROVO A VEDERE SE SONO PRESENTI ALTRI RECORDS DEGLI UTENTI A CUI MANCANO NOMINATIVE E ADDRESS
v = df['user_code'].value_counts()
v[v.gt(1)].index.compute()

StringIndex(['3A3280EB6C0F4C672AB923ECC5CB01D29BE9FBCF'
 'FA5E3ADB2DE4D8651BA10F9E2A6E2F468EA359BA'
 'C1D72C62ED72FB72133E706FF5D7F84BF28A9C40' ...
 '41CE9ABCDE10249C5090C866E035DF5788B64772'
 'E3F9978701B543A3E84EF555D895C021739C5268'
 '1A81211D36C8A6D53D1BCB4D0426EF75B210AD5D'], dtype='object')

In [16]:
# DATASET CON GLI user_code CHE COMPAIONO PIù DI UNA VOLTA
df[df['user_code'].isin(v[v.gt(1)].index.compute())].head()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>


In [15]:
# Le date potrebbero fornirci informazioni sulla billing_frequency, o viceversa, come lo stabiliamo?
# Cosa indica la colonna dell'emission_date??

#NOTA PER DASK, SE FACCIAMO 0 CI TORNA IL PRIMO RECORD DI TUTTE LE 52 PARTIZIONI
df.loc[0, 'billing_frequency'].compute()

0         <NA>
0     bimester
0     bimester
0     bimester
0     bimester
0    quarterly
0    quarterly
0      monthly
0     bimester
0     bimester
0     bimester
0     bimester
0     bimester
0     bimester
0         <NA>
0     bimester
0     bimester
0     bimester
0     bimester
Name: billing_frequency, dtype: string

## Locate Outliers DA PORTARE 

In [22]:
base['F1_kWh'].astype(np.float64).quantile(0.1).compute()

0.0

In [6]:
#CERTE COLONNE CON IL TIPO DI DATO PARTICOLARE TIPO FLOATDTYPE64 VANNO CONVERTITE IN TIPI COMPRENSIBILI
base.get_df()['F1_kWh'] = base.get_df()['F1_kWh'].astype(float)
base.locate_outliers('F1_kWh').head()

6       -1.0
44    1356.0
52     -15.0
61     -30.0
71    6513.0
Name: F1_kWh, dtype: float64

In [17]:
type = df.dtypes
num = type[type!='object'][type!='bool']
num = num.index.to_list()
num

['bill_id',
 'F1_kWh',
 'F2_kWh',
 'F3_kWh',
 'address',
 'nominative',
 'age',
 'gas_amount',
 'gas_average_cost',
 'light_average_cost',
 'gas_consumption',
 'light_consumption',
 'gas_offer',
 'light_offer',
 'light_amount',
 'average_unit_light_cost',
 'average_light_bill_cost',
 'average_unit_gas_cost',
 'average_gas_bill_cost',
 'billing_frequency',
 'gas_system_charges',
 'light_system_charges',
 'gas_material_cost',
 'light_transport_cost',
 'gas_transport_cost',
 'light_material_cost']

In [8]:
tmp = base.locate_outliers("F1_kWh")
tmp
# a lot of task per partition

Dask Series Structure:
npartitions=57
    float64
        ...
     ...   
        ...
        ...
Name: F1_kWh, dtype: float64
Dask Name: index, 743 tasks

In [9]:
tmp = tmp.persist()
tmp
# no more than 1 task per partition

Dask Series Structure:
npartitions=57
    float64
        ...
     ...   
        ...
        ...
Name: F1_kWh, dtype: float64
Dask Name: index, 57 tasks

In [ ]:
tmp.shape

In [ ]:
tmp = ddf.locate_outliers("F1_kWh")
tmp = tmp.compute()
tmp.shape

In [38]:
for col in num:
    tmp = base.locate_outliers(col)
    tmp = tmp.compute()
    
    print(f"{col} has {tmp.shape[0]} outliers")

  6%|▋         | 1/16 [00:09<02:21,  9.43s/it]

bill_id has 2152864 outliers


 12%|█▎        | 2/16 [00:17<02:04,  8.89s/it]

F1_kWh has 149572 outliers


 19%|█▉        | 3/16 [00:26<01:51,  8.56s/it]

...

light_transport_cost has 1338296 outliers


 94%|█████████▍| 15/16 [02:19<00:09,  9.50s/it]

gas_transport_cost has 1867752 outliers


100%|██████████| 16/16 [02:27<00:00,  9.24s/it]

light_material_cost has 1037563 outliers


## Sort Data DA PORTARE (qui c'è già roba con dask, da verificare se funziona)

In [7]:
#CERTE COLONNE CON IL TIPO DI DATO PARTICOLARE TIPO FLOATDTYPE64 VANNO CONVERTITE IN TIPI COMPRENSIBILI
base.get_df()['F1_kWh'] = base.get_df()['F1_kWh'].astype(float)
base.sort("F1_kWh").tail()
# working

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
461174,763206,NaN,91.0,99.0,MONTECCHIO,4184A9CEE19BB60C749D312255E3293A6CAB3511,7FE5F42763C8557BDEA2BF17B57630C25DE5F595,M,52,D22B2CEA63C59A32A7473C8173406541A106496A,...,<NA>,0DB35BBC1052B5067932A45FD482AFF81A2DF377,bimester,False,<NA>,-87.9,<NA>,-99.88,<NA>,38.79
469346,771378,NaN,107.0,<NA>,FOSSALTA DI PIAVE,BFF5D868FCA1CFAB9441639F0888566273833E61,3156EF3B3D32BD82F3AE3EF1B4115921E9C8F1F9,M,42,D23AC25D43B729E42A0FD415D00C92EBF5E467F8,...,<NA>,E117BD1DB181B0E522B38856EDDE11A90BED108C,bimester,False,<NA>,11.41,<NA>,16.48,<NA>,42.71
399408,1257944,NaN,63.0,<NA>,VIGONZA,62CD6ADF5A4B937AF85731F6729EB0879255A2CA,F8CDF6A8EA52B5BA0F9133161E5C75C98A21A5DD,F,26,C71C5E4FA06FA3D77A434A191D78F4A45B12CB34,...,<NA>,F5E33F8093411ABFDA555AD3E1A013EF5CEF3A67,bimester,False,<NA>,6.02,<NA>,15.32,<NA>,25.88
275461,2245832,NaN,<NA>,<NA>,MALCESINE,B850C7CB5B5A1669380B01658222835FB07D4514,D3043780957BF872B4A597E50CA9162182B11B60,M,46,66FE66C26F9A0D640F8FF52872BC7027DFF4337E,...,<NA>,1B1A9FA51C8BAC5A23BCFA2AB551510D630A6073,<NA>,False,0.0,0.13,0.0,0.03,0.0,0.38
402530,2372901,NaN,<NA>,<NA>,CORNUDA,E240ACE1E0BC1D105DDC68332F0765259095BBA8,DB131C242DB9548CD4F3780A5195ABF4B0933DB4,F,56,CC92C63F1B30E4987638A0EB7959CC16BE116AA4,...,<NA>,088B6416D4900E66C52D184D73094351D87D1560,<NA>,False,<NA>,21.11,<NA>,14.15,<NA>,18.91


### Bazze mirco

In [ ]:
tmp = ddf.sort_index("F1_kWh").compute()
# not working

In [ ]:
idx = tmp.index.to_arrow().to_pylist()
# too mutch long list

In [ ]:
tmp.merge(ddf.df, on=["F1_kWh"], how="left")

# 2. VALIDATION

In [ ]:
''' 
+++++ DATA VALIDATION +++++
    • Check data range
    • Check column uniqueness
    • Find data-mismatched data types
'''

## Check Data Range DA PORTARE TUTTE (mi tirava fare un sottogruppo)

### Age nulla per PIVA

In [8]:
# Controllo se chi ha P ha età nulla
import pandas as pd
df['age'] = df['age'].mask((df['sex']=='P') & (df['age']>=0), np.nan)
df[(df['sex']=='P')][['sex','age']].sample(frac=0.1).compute()

,sex,age
64385,P,NaN
45634,P,NaN
70015,P,NaN
116608,P,NaN
129934,P,NaN
...,...,...
536332,P,NaN
534336,P,NaN
534187,P,NaN
508897,P,NaN


### Age >= 18

In [10]:
# Age >= 18
# Age non ha valori nulli (già controllato in precedenza IN TEORIA)
# Dovrebbe corrispondere al numero di PIVA
df['age'].isna().sum().compute()

401212

In [88]:
# Che fare con i minorenni? Li mettiamo tutti a 18 anni?
df[(df['age'] < 18)].shape[0].compute()

111689

### Translation in English

In [12]:
df['supply_type'].value_counts(dropna=False).compute()

gas           5686288
luce          3684510
gas e luce    1126345
Name: supply_type, dtype: int64

In [13]:
# Dobbiamo metterli in inglese? Si può fare con un map
ing = {
    'luce': 'light',
    'gas e luce': 'gas and light',
    'gas' : 'gas',
}
df['supply_type'] = df['supply_type'].map(ing)

In [14]:
df['supply_type'].value_counts(dropna=False).compute()

gas              5686288
light            3684510
gas and light    1126345
Name: supply_type, dtype: int64

### Data control

### billing_freq & bill_type Analysis

In [18]:
# Che fare qua? Billing frequency (‘monthly’, ‘quarterly’, etc.)
# Si potrebbero calcolare
df['billing_frequency'].value_counts(dropna=False).compute()

bimester     7611046
quarterly    1386049
NaN           784810
monthly       715238
Name: billing_frequency, dtype: Int64

In [20]:
# Kind of invoice (False means a “standard bill”)
df['bill_type'].value_counts(dropna=False).compute()

False                     10446508
variation note               30477
change note correction       19616
cancellation bill              542
Name: bill_type, dtype: int64

### light_offer_type Analysis

In [11]:
# Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
df['light_offer_type'].value_counts(dropna=False).compute()

light                5712814
light single zone    4413764
ligth bizone          189594
light multi zones     180971
Name: light_offer_type, dtype: int64

In [23]:
# I 2 valori di light mi puzzano, famo uno strip?
df['light_offer_type'] = df['light_offer_type'].str.strip()
df['light_offer_type'].value_counts(dropna=False).compute()
# GHAVEVO RAGIONE Vè

light                5712814
light single zone    4413764
ligth bizone          189594
light multi zones     180971
Name: light_offer_type, dtype: int64

## Check Column Uniqueness DA PORTARE E DA VERIFICARE

In [ ]:
# The story changes here, we got these pairs as candidates as duplicate columns 
# (those which have same values for each row.)

# ATTENZIONE!! Probabile che le date del gas siano invertite, trovare un modo per confermarlo

In [12]:
## Sserve il date formatter!!
df['emission_date'].head()

0   2020-12-31
1   2020-12-31
2   2020-12-31
3   2020-12-31
4   2020-12-31
Name: emission_date, dtype: datetime64[ns]

## Find Data Mismatch

In [5]:
# VERY time consuming, expecially with DASK, with DASK_CUDF It is a lot faster but It takes a lot of time as well
# base.get_duplicate_columns()
# Ne vengono individuate solo 5 di coppie 100% fedeli, 
# ma cambia la questione se abbassiamo la soglia? Ci sono degli errori sopra perchè 
# quelle 100% uguali non vengono riconosciute, mentre qua sì
duplicates = [('date', 'light_start_date'),('date', 'gas_end_date'),('light_start_date', 'gas_end_date'),('light_end_date', 'gas_start_date'),('gas_average_cost', 'average_unit_gas_cost')]
duplicates

[('date', 'light_start_date'),
 ('date', 'gas_end_date'),
 ('light_start_date', 'gas_end_date'),
 ('light_end_date', 'gas_start_date'),
 ('gas_average_cost', 'average_unit_gas_cost')]

In [ ]:
df['date'] == df['light_start_date']

In [16]:
base['tv'].head()

0    0,00
1    0,00
2    0,00
3    0,00
4    0,00
Name: tv, dtype: object

# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #

In [49]:
#poichè probabilmente con i <NA> ho un sacco di problemi "lasciatemi castaareeee.. 
#con la chitarra in maaanoooo, lasciatemi castare.. sono un italiano"

df = base.fill_nan(np.nan)
df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,Float64,Float64,object,string,string,object,int64,object,object,object,object,object,string,string,Float64,object,object,object,object,object,Float64,Float64,uint64,object,uint64,object,object,string,string,string,string,string,object,string,object,Float64,Float64,Float64,Float64,Float64,Float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [50]:
# setto le colonne con i tipi di dato in mappone
df = base.cast_columns_types(mappone)

In [51]:
df['emission_date']

Dask Series Structure:
npartitions=10
    datetime64[ns]
               ...
         ...      
               ...
               ...
Name: emission_date, dtype: datetime64[ns]
Dask Name: getitem, 1589 tasks

In [ ]:
# Abbiamo dei valori non castabili a float nelle colonne interessate
# Vediamo cos'ho nelle colonne str
#print(string_cols)
for c in string_cols:
    print("Col: ", c)
    cosa = df[c].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    print(cosa)

In [ ]:
# C'è della merda, ottimizzo la cosa riducendo il campo d'analisi a solo le colonne float64 di mappone
mappone_float64_cols = []

for (k, v) in mappone.items():
    if v == 'float64':
        mappone_float64_cols.append(k)

final_float_cols = [x for x in mappone_float64_cols if x not in float_cols]
mappone_float64_cols
float_cols
final_float_cols = [x for x in final_float_cols if x not in ['gas_offer']]

In [ ]:
# Altro caso di colonna dal dubbio gusto
df['gas_offer']

In [ ]:
# Vediamo cos'ho nelle colonne str
#print(string_cols)
lista_rename = []
for c in final_float_cols:
    cosa = df[c].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    if cosa != '0.00' and cosa != 'nan':
        print("Col: ", c + ' ' + cosa)
        print(cosa)
        p = {
            c : c + ' ' + cosa
        }
        lista_rename.append(p)
        df[c] = df[c].str.replace(',', '.').str.extract('-?([0-9]*\.[0-9]+)').astype(float)
df

In [ ]:
# voglio trovare le colonne str che diventeranno float che generano problemi
# VIRGOLE DI MERDA
problematiche = [x for x in string_cols if x in final_float_cols]
for p in problematiche:
    print(p)
    df[p] = df[p].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    print(float(df[p][0]))

In [ ]:
float(0.48)

In [ ]:
# Ho diverse colonne da cui eliminare le unità di misura...
df['gas_average_cost'].dtype

In [ ]:
# in 'bill_type' False == 'standard bill'
# Funziona solo se la funzione sopra non va in errore.

df = base.replace('bill_type', 'False', 'standard bill', False)
df

In [ ]:
#Verifica della bontà della sostituzione
df['bill_type'].value_counts()

In [ ]:
# Non c'è nessuna riga tutta nulla, Giulio è contento
df.dropna(how='all')

# 3. STRUCTURING

In [ ]:
''' 
+++++ DATA STRUCTURING +++++
    • Change column data types
    • Delete, split or merge columns
    • Pivot and unpivot
'''

## Change Column Data Types 

In [ ]:
# aggregabile con lo step precedente
# L'HO FATTO SOPRA, è DA ORDINARE E PERFEZIONARE

## Delete, Split or Merge Columns

In [ ]:
# rimozione colonne duplicate 

## Pivot and Unpivot

In [ ]:
# non saprei cosa farci

# 4. ENRICHMENT

In [ ]:
''' 
+++++ DATA ENRICHMENT +++++
    • Calculate columns using expressions
    • Set primary key column
    • Join or append DataFrames
    • Group by and aggregate records
    • Scale column values into a certain range
    • Encode categorical data (one-hot encoding and label encoding)
'''

## Calculate Columns Using Expressions

## Set PK column

# 5. FILTERING

In [ ]:
''' 
+++++ DATA FILTERING +++++
    • Sample rows
    • Select a subset of rows through a user-defined query
'''

## Sample Rows

In [ ]:
df.sample(20)

In [ ]:
# SELECT A SUBSET OF ROWS?!?

# 6. CLEANING

In [ ]:
''' 
+++++ DATA CLEANING +++++
    • Change data format and case
    • Deduplicate data
    • Missing value imputation
    • Find and replace values
'''

In [ ]:
# CHANGE DATA FORMAT AND CASE
# data format dovrebbe già essere fatto a questo punto
# case pure se vogliamo aver fatto tutto per bene

In [ ]:
# DEDUPLICATE DATA
# controllo se ho più bollette uguali dello stesso cliente

In [ ]:
ddf

In [ ]:
# MISSING VALUE IMPUTATION
# qua? Spero di averne pochissimi nel caso

In [ ]:
# FIND AND REPLACE VALUES
# ossia?

In [ ]:
x = np.nan
float(x)